<a href="https://colab.research.google.com/github/pacificspatial/flateau/blob/main/notebook/sdsc_bootcamp_tokyo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PLATEAUではじめる空間クエリ
by Hiroo Imaki (PSS) 2023/09/29

## はじめに
このSpatial Data Science の Bootcampでは、空間データサイエンスを始めるにあたり、一番重要なスキルである、空間クエリの第一歩について学びます。そのために、最近進歩が目覚ましい、DuckDBとその拡張機能を利用します。DuckDBは、コマンドラインで利用できるだけでなく、Pythonでも簡単にインストールして利用できます。解析するデータは、国交相が進めるPLATEAUの建物データにしました。建物データは元々3DのCityGML形式でオープンデータとして配布されていますが、今回はそのデータをGeoParquet形式にあらかじめ変換したデータを用意しました。

DuckDBは、2019年に開発が始まって、今年空間解析の機能拡張が追加された空間解析を行う人にとってはとても新しいツールです。
これまでは、PostgreSQLやGoogle BigQueryを使って空間データを解析することが多かったのですが、ローカル環境で大きな空間データを解析できるDuckDBは、空間解析を行う人は使えるようになっていたいツールの一つだと思います。

今回のチュートリアルは、SQLの基礎的な知識がある方で、空間データの解析に興味のある方が対象となっています。SQLについてよくわからないという方は、https://www.w3schools.com/sql/ を初めたくさんのサイトがあるので、そちらを参照してください。また、前提として、PythonがNotebook環境で利用できる方を想定しています。ただし、なんの知識がなくても、このノート自体でチュートリアルが完結しているので、上から順番にセルを実行していただければ、手を動かしながら空間クエリについて学べるようになっています。学習に役立つサイトへのリンクも各所に記載しました。

皆さんが、空間クエリを活用して、さまざまな空間的な課題を解決するのに少しでもお役に立てば幸いです。

## 空間クエリとは
![SQLのイメージ](https://github.com/pacificspatial/flateau/blob/main/notebook/image/sql_screenshot.png?raw=1)

SQLとは、Structured Programming Language (シークゥオル、またはエスキューエルと読むことが多い）の略称で、リレーショナルデータベース(RDBMS)に格納されたデータを管理またデータに対し問い合わせするための言語です。文法は、データベースごとに（例えば、PostgreSQL/PostGIS、Bigqery SQL、Snowflake SQL, DuckDB）多少異なることがありますが、大きな違いはないため、 一度学ぶと、比較的簡単に他のデータベースでもクエリを書くことができます。

SQLを頻繁に使う人は、データエンジニア、データサイエンティスト、データ解析を行う人などです。

SQLを学ぶポイント：いわゆる　"the big 6" コマンドと呼ばれる、SELECT, FROM, WHERE, GROUP BY, HAVING, ORDER BY が文法の骨格です。SQLを書くときは、SQLコマンドは、できれば大文字で書くのが推奨されていますが、必須ではありません。空間クエリに関わるコマンドは、ST_で始まることが多く、代表的なものとしては、ST_BUFFER, ST_CENTERMEDIAN, ST_POLYGONIZE などがあります。実装されているコマンドの数は、データベースごと異なります。ちなみに、ST_は Spatial & Temporal の略です。

* [PostgreSQL/PostGIS](http://cse.naro.affrc.go.jp/yellow/pgisman/2.2.0/reference.html) 約260（ST関数以外も含む）
* [Google BigQuery](https://cloud.google.com/bigquery/docs/geospatial-data) 66
* [Snowflake](https://docs.snowflake.com/en/sql-reference/functions-geospatial) 66
* [DuckDB](https://duckdb.org/docs/extensions/spatial.html) 44


### 空間クエリを学べるリソース:
CARTOのブログには、空間クエリに関する詳しいチュートリアルがあります
 * [5 maps you didn't know you could create with SQL](https://carto.com/blog/create-maps-dataviz-with-sql)
 * [Ultimate Guide to Spatial Joins & Predicates with SQL](https://carto.com/blog/guide-to-spatial-joins-and-predicates-with-sql)
 * [空間クエリチートシート](https://drive.google.com/file/d/1ncaA_CES-lC4ZqzZ7iC__8kDpED9ieFr/view)

今日のプレゼンターのHelen McKenzie が既存のGISユーザ向けに[空間クエリの始め方についての記事](https://www.helenmakesmaps.com/post/how-to-sql-a-guide-for-gis-users)を書いています。

[University of Washington のサイト](https://uwescience.github.io/SQL-geospatial-tutorial/?utm_content=239999238&utm_medium=social&utm_source=linkedin&hss_channel=lcp-5084329)もチュートリアルや演習を通して空間SQLを学べるようになっています。

以上は、CARTOのSpatial Data Science Bootcampsのハンドブックからの抜粋たもので、一部このハンズオンのために内容を改変し日本語に翻訳しています。

また、以下がこのハンズオンの資料を作ったり勉強したりした時に使ったサイトや動画です。
* [Matt Forrest のYouTube動画](https://www.youtube.com/watch?v=ljzpm3Mrw-I&t=1426s)
* [Mark Litwintschik のブログ](https://tech.marksblogg.com/duckdb-gis-spatial-extension.html)
* [Mother Duck YouTube チャンネル](https://www.youtube.com/@motherduckdb)
* [DuckDB Spatial Github Repo](https://github.com/duckdblabs/duckdb_spatial)
* [DuckDB Spatial 機能拡張](https://duckdb.org/docs/extensions/spatial#spatial-copy-functions)
* [DuckDB Spatial ブログ](https://duckdb.org/2023/04/28/spatial.html)
* [Google BigQuery のGIS機能](https://cloud.google.com/bigquery/docs/geospatial-data)


## PLATEAUと都市のデジタルツインについて

[PLATEAU](https://www.mlit.go.jp/plateau/)とは、国土交通省が主導する、日本全国の3D都市モデルの整備・活用・オープンデータ化プロジェクト。2021年度、全国56、2022年度は新たに71、合計127都市の3D都市モデルデータを[CityGML形式でオープンデータとして公開](https://www.geospatial.jp/ckan/dataset/plateau)

[東京都デジタルツイン実現プロジェクト](https://info.tokyo-digitaltwin.metro.tokyo.lg.jp)は、都市のデジタルツインの実装を目指したプロジェクト。PLATEAUのデータも取り込みつつ、東京都の課題解決と都民のQOL向上を目指す。

### 都市のデジタルツインと空間解析
都市のデジタルツインの一つの要は、空間データを活用し、市民への情報を提供したり、課題を解決すること。そのためには、空間データ解析がとても重要な位置を占める。都市モデルが準拠するCityGML形式のデータは、もともと、さまざまな目的合わせ、変換して利用するためのデータフォーマットであるため、可視化のためには、例えば、3D Tiles形式に変換したり、解析のためには、空間解析に向いたデータ形式に変換して利用する必要がある。

* CityGMLの可視化 ---> 3D Tiles
* CityGMLの解析　 ---> GeoParquet, GeoPackage, Shapefile, etc

ただし、データの変換には、今のところ専門的な知識が必要とされるのが現状。

### CityGMLの変換
なので、

[FME](https://www.safe.com)で[GeoParquet形式とGeoPackage形式に変換しました](https://github.com/pacificspatial/flateau)。FMEを使ったPLATEAUデータ変換の概要は、[このサイトで](https://www.mlit.go.jp/plateau/learning/tpc04-1/)。

今後は変換した成果だけではなく、[FMEのワークスペースもGithubからアクセスできるようにしたい](https://github.com/pacificspatial/flateau/tree/main/workspace)と思っています。



##  ハンズオンの概要と準備
### ソフトウェアとデータ
ハンズオンでは、DuckDBを使って、GeoParquet形式に変換したPLATEAUの都市モデルデータを検索、解析します。
DuckDBを選んだのにはいくつか理由があります。
* インストールが簡単（通常のDBMSのようにサーバ・クライアントの構成ではない）
* 無茶苦茶早い
* 空間解析のための関数が最近実装
* GeoParquetが使える
* ネイティブPOINT_2D, BOX_2Dジオメトリ
* ベクタ化された実行モデルと列指向データ保存形式 ---> データを効率的クエリできる
* OLAP (online analytical processing)　---> 大量のデータに対して高速で多次元分析を行うソフトウェア。複数のリレーショナル・データ・セットからデータを抽出し、それを多次元形式に再編成することで、非常に高速な処理と非常に洞察に富んだ分析を実現
* インプロセスデータベースエンジン ---> クライアントーサーバ構成ではない。従来のサーバ経由のアクセス時間や通信時間のロスを減らせます．しかも，システム全体を統一的に管理でき，高可用性（システム・ダウンを起こさず，連続稼働する機能）を実現しやすい

DuckDBの位置付けに関するまとめは、[DuckDB入門](https://zenn.dev/notrogue/articles/1193d0ab8d8eda) にうまくまとめられていますので、参考にしてください。

GeoParquetを推すのは、いくつか理由があります
* Cloud Native フォーマットで、データサイズがとても小さくなる
* 最近QGISやFMEでも読み書きができるようになった
* Chris Holms、Javier de la Torre、Tim Shuab、Matt Forrest がいいと言っている

ただし、DuckDBは現時点では、以下の点で制限があります（対応予定ではある）。
* GiSTなどの空間インデックスに対応していない
* GEOMETRYタイプの仕様がフル実装されていない（ZやM、サブタイプなど）。GEOGRAPHYタイプに対応していない
* GEOGRAPHYタイプに対応していないので、緯度経度を使った演算（面積計算など）ができない
* PostGISに比べ利用できる関数がまだ少ない
* GeoParquetをテイティブでサポートしていない（[する予定](https://github.com/duckdblabs/duckdb_spatial/issues/27)）

# ハンズオン
### それでは実際にDuckDBを使ってみます。

In [2]:
# 最初にハンズオンで使うライブラリをインストール
# データを地図として見るためのライブラリ
!pip install leafmap
!pip install pydeck

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.6/175.6 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.1/72.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: jsonschema
    Found existing installation: jsonschema 4.19.0
    Uninstalling jsonschema-4.19.0:
      Successfully uninstalled jsonschema-4.19.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 17.1 MB/s eta 0:00:00


In [3]:
# ライブラリのインポート
# Duckdb is already in Colab!
import duckdb
import pandas as pd
import geopandas as gpd
import leafmap
from shapely import wkt

## DuckDBが動くかテスト

In [5]:
# インメモリデータベースを使ってクエリが実行できるかテスト
duckdb.sql('SELECT 42')

┌───────┐
│  42   │
│ int32 │
├───────┤
│    42 │
└───────┘

# DuckDBの基本操作

In [6]:
# リレーションの確認。クエリの結果を変数に格納して、他のクエリで呼び出せる。
r1 = duckdb.sql('SELECT 42 AS i')
duckdb.sql('SELECT i * 2 AS k FROM r1').show()

┌───────┐
│   k   │
│ int32 │
├───────┤
│    84 │
└───────┘



In [7]:
# インメモリにデータベースを作成。このノートを終了したらメモリから消える。
duckdb.sql('DROP TABLE IF EXISTS ducks;')
duckdb.sql("CREATE TABLE ducks AS SELECT 3 AS age, 'mandarin' AS breed;")

# DuckDBのスペシャルクエリ！
duckdb.sql("FROM ducks;")

┌───────┬──────────┐
│  age  │  breed   │
│ int32 │ varchar  │
├───────┼──────────┤
│     3 │ mandarin │
└───────┴──────────┘

In [18]:
# インメモリではなく、ファイルとしてデータを格納したいとき。この例では、test.dbというデータベースを作成する。
# データベース名、拡張子はなんでも良い。拡張子がなくても良い。
# 以下ではコネクションを作成しているが、デフォルトの設定で作成したもの、各種オプションを明示的に指定したもの、
# 外部データベースを指定したもの、インメモリを指定したもの、などの組み合わせで示した。

# デフォルトでの書き方
con = duckdb.connect('test.db')

# オプションで、allow_unsigned_extensionsをtrueにすると、公式ではないH3拡張機能も読み込めるようになる。
#con = duckdb.connect(database="test.db", read_only=False, config={"allow_unsigned_extensions": "true"});

# in-memoryの場合は以下のように書く
#con = duckdb.connect(database=":memory:", read_only=False, config={"allow_unsigned_extensions": "true"});


In [20]:
# テーブルを準備
con.sql('DROP TABLE IF EXISTS junk')
con.sql('CREATE TABLE junk(i INTEGER)')
con.sql('INSERT INTO junk VALUES (42)')
# テーブルを表示
con.table('junk').show()

┌───────┐
│   i   │
│ int32 │
├───────┤
│    42 │
└───────┘



In [96]:
# 接続を閉じる
con.close()

NameError: ignored

In [11]:
# 外部のファイルを読み込む。csv, parquet, json, Pandas DataFrame, Arrow objectに対応
# 読み込みのための関数を使うと以下のようにファイルを読み込める
#duckdb.read_csv('sample_data/mnist_test.csv')
#duckdb.read_parquet('example.parquet')
duckdb.read_json('sample_data/anscombe.json')


┌─────────┬────────┬────────┐
│ Series  │   X    │   Y    │
│ varchar │ double │ double │
├─────────┼────────┼────────┤
│ I       │   10.0 │   8.04 │
│ I       │    8.0 │   6.95 │
│ I       │   13.0 │   7.58 │
│ I       │    9.0 │   8.81 │
│ I       │   11.0 │   8.33 │
│ I       │   14.0 │   9.96 │
│ I       │    6.0 │   7.24 │
│ I       │    4.0 │   4.26 │
│ I       │   12.0 │  10.84 │
│ I       │    7.0 │   4.81 │
│ ·       │     ·  │     ·  │
│ ·       │     ·  │     ·  │
│ ·       │     ·  │     ·  │
│ IV      │    8.0 │   5.76 │
│ IV      │    8.0 │   7.71 │
│ IV      │    8.0 │   8.84 │
│ IV      │    8.0 │   8.47 │
│ IV      │    8.0 │   7.04 │
│ IV      │    8.0 │   5.25 │
│ IV      │   19.0 │   12.5 │
│ IV      │    8.0 │   5.56 │
│ IV      │    8.0 │   7.91 │
│ IV      │    8.0 │   6.89 │
├─────────┴────────┴────────┤
│    44 rows (20 shown)     │
└───────────────────────────┘

In [14]:
# 外部のファイルを読み込む。SQL内に直接ファイルパスを書くこともできる
duckdb.sql('SELECT * FROM "sample_data/anscombe.json"')

┌─────────┬────────┬────────┐
│ Series  │   X    │   Y    │
│ varchar │ double │ double │
├─────────┼────────┼────────┤
│ I       │   10.0 │   8.04 │
│ I       │    8.0 │   6.95 │
│ I       │   13.0 │   7.58 │
│ I       │    9.0 │   8.81 │
│ I       │   11.0 │   8.33 │
│ I       │   14.0 │   9.96 │
│ I       │    6.0 │   7.24 │
│ I       │    4.0 │   4.26 │
│ I       │   12.0 │  10.84 │
│ I       │    7.0 │   4.81 │
│ ·       │     ·  │     ·  │
│ ·       │     ·  │     ·  │
│ ·       │     ·  │     ·  │
│ IV      │    8.0 │   5.76 │
│ IV      │    8.0 │   7.71 │
│ IV      │    8.0 │   8.84 │
│ IV      │    8.0 │   8.47 │
│ IV      │    8.0 │   7.04 │
│ IV      │    8.0 │   5.25 │
│ IV      │   19.0 │   12.5 │
│ IV      │    8.0 │   5.56 │
│ IV      │    8.0 │   7.91 │
│ IV      │    8.0 │   6.89 │
├─────────┴────────┴────────┤
│    44 rows (20 shown)     │
└───────────────────────────┘

In colab, a multiple line edit/ a mass edit hortcut.
1. Alt + Click for Windows
2. Option + Click for MacOS

And to select all occurrences of the existing selection, use:

1. Ctrl + Shift + L for Windows
2. Command + Shift + L for MacOS

In [16]:
# 読み込んだデータをDataFrameなどに格納もできる
#duckdb.sql('SELECT * FROM "sample_data/anscombe.json"').fetchall()   # Python objects
junk_df = duckdb.sql('SELECT * FROM "sample_data/anscombe.json"').df()         # Pandas DataFrame
#duckdb.sql('SELECT * FROM "sample_data/anscombe.json"').pl()         # Polars DataFrame
#duckdb.sql('SELECT * FROM "sample_data/anscombe.json"').arrow()      # Arrow Table
#duckdb.sql('SELECT * FROM "sample_data/anscombe.json"').fetchnumpy() # NumPy Arrays

junk_df.head()

,Series,X,Y
0,I,10.0,8.04
1,I,8.0,6.95
2,I,13.0,7.58
3,I,9.0,8.81
4,I,11.0,8.33


## Extension を確認

In [26]:
# 拡張機能の確認。デフォルトでは、httpfs, spatialなどは読み込まれていない。
duckdb.sql('select * from duckdb_extensions()')

┌──────────────────┬─────────┬───────────┬──────────────────────┬──────────────────────────────────┬───────────────────┐
│  extension_name  │ loaded  │ installed │     install_path     │           description            │      aliases      │
│     varchar      │ boolean │  boolean  │       varchar        │             varchar              │     varchar[]     │
├──────────────────┼─────────┼───────────┼──────────────────────┼──────────────────────────────────┼───────────────────┤
│ autocomplete     │ false   │ false     │                      │ Add supports for autocomplete …  │ []                │
│ fts              │ true    │ true      │ (BUILT-IN)           │ Adds support for Full-Text Sea…  │ []                │
│ httpfs           │ true    │ true      │ /root/.duckdb/exte…  │ Adds support for reading and w…  │ [http, https, s3] │
│ icu              │ true    │ true      │ (BUILT-IN)           │ Adds support for time zones an…  │ []                │
│ inet             │ false   │ f

## Extensionをインストールして読み込む

In [4]:
# HTTPS経由でファイルを読み込むための機能と、空間データ取扱のための機能を読み込む
duckdb.sql("INSTALL 'httpfs'");
duckdb.sql("INSTALL 'spatial'");

duckdb.sql('LOAD httpfs')
duckdb.sql('LOAD spatial')

## 読み込める空間データファイルフォーマットを確認

In [95]:
# ドライバを確認。なんとGeoParquetはサポートされていない。でもParquetはサポートされているので大丈夫！
# そのうちサポートされるらしい。https://github.com/duckdblabs/duckdb_spatial/issues/27
duckdb.sql('SELECT * FROM st_drivers()').df()

InvalidInputException: ignored

# ハンズオンのデータをDuckDBで取り扱う

## Geometryを取り扱えるか確認

In [9]:
duckdb.sql('SELECT ST_POINT(0,0)')

┌────────────────┐
│ st_point(0, 0) │
│    geometry    │
├────────────────┤
│ POINT (0 0)    │
└────────────────┘

## 今回使うデータのリスト
* [建物データについて](https://github.com/pacificspatial/flateau/tree/main/data/plateau)
  * [建物セントロイド](https://flateau.s3.ap-northeast-1.amazonaws.com/data/plateau/tokyo23/2022/buildings/tokyo23_2022_buildings_centroid.parquet)
  * [建物フットプリントポリゴン](https://flateau.s3.ap-northeast-1.amazonaws.com/data/plateau/tokyo23/2022/buildings/tokyo23_2022_buildings_polygon.parquet)
  * [洪水浸水想定：属性テーブル](https://flateau.s3.ap-northeast-1.amazonaws.com/data/plateau/tokyo23/2022/buildings/tokyo23_2022_buildings_risk_flooding.parquet)
  * [高潮浸水想定：属性テーブル](https://flateau.s3.ap-northeast-1.amazonaws.com/data/plateau/tokyo23/2022/buildings/tokyo23_2022_buildings_risk_high_tide.parquet)
  * [地滑り：属性テーブル](https://flateau.s3.ap-northeast-1.amazonaws.com/data/plateau/tokyo23/2022/buildings/tokyo23_2022_buildings_risk_land_slide.parquet)
* [地形データについて](https://github.com/pacificspatial/flateau/blob/main/data/topography.md)
  * [標高 H3 Level10 ポリゴン](https://flateau.s3.ap-northeast-1.amazonaws.com/data/topography/tokyo23_elevation_h3lvl10.parquet)
  * [斜面傾斜 H3 Level10 ポリゴン](https://flateau.s3.ap-northeast-1.amazonaws.com/data/topography/tokyo23_slope_h3lvl10.parquet)

データを視覚化してみたいときは、WindowsならQGISの最新版でGeoParquetがそのままで表示できます。

Macでは、最新版のQGISインストーラでインストールされるQGISではまだGeoParquetが読めないので、[この記事の手順](https://atsik-ai.com/posts/qgis-geoparquet)に従ってQGISをインストールすると読めます。


## ハンズオン用のデータを読み込む


*   GeoParquetは [ST_READ()](https://duckdb.org/docs/extensions/spatial#spatial-table-functions) ではそのまま読めないので、Parquetとして読む
*   読み込む際に、[ST_GeomFromWKB(geometry)](https://duckdb.org/docs/extensions/spatial#geometry-conversion) で blob で読み込まれるGeometryを変換する
*   Geometryを [DuckDBのネイティブジオメトリタイプ(POINT_2D等)](https://duckdb.org/docs/extensions/spatial.html) にキャストとして読み込むこともできる。ただし、[ネイティブタイプに対応した関数](https://github.com/duckdblabs/duckdb_spatial#supported-functions)はまだ限られている。




In [94]:
# 建物2Dポリゴンの読み込み。時間がかかると思う。httpfs 機能拡張を読み込んだので、インターネット経由でデータが読み込める
polygon_file = 'https://flateau.s3.ap-northeast-1.amazonaws.com/data/plateau/tokyo23/2022/buildings/tokyo23_2022_buildings_polygon.parquet'

duckdb.sql(
    "CREATE OR REPLACE TABLE building_polygon AS SELECT id, gml_id, h3index10, cal_zmin_m, cal_area_m2, \
    cal_height_m::double cal_height_m, ST_GeomFromWKB(geometry) AS geom FROM '{}' ".format(polygon_file)
    )

InvalidInputException: ignored

In [20]:
duckdb.sql('SELECT * FROM building_polygon LIMIT 3')

┌───────┬──────────────────────┬────────────────────┬───┬───────────────────┬───────────────────┬──────────────────────┐
│  id   │        gml_id        │     h3index10      │ … │    cal_area_m2    │   cal_height_m    │         geom         │
│ int64 │       varchar        │       uint64       │   │      double       │      double       │       geometry       │
├───────┼──────────────────────┼────────────────────┼───┼───────────────────┼───────────────────┼──────────────────────┤
│     1 │ bldg_fc50c7d9-76ac…  │ 622329813002289151 │ … │ 40.44564363531246 │ 6.734999999999999 │ POLYGON ((139.7123…  │
│     2 │ bldg_2e11a57e-c315…  │ 622329810539020287 │ … │ 53.06161553075292 │             9.833 │ POLYGON ((139.7071…  │
│     3 │ bldg_72de059a-ddf0…  │ 622329810539118591 │ … │ 64.26319241025253 │             6.605 │ POLYGON ((139.7065…  │
├───────┴──────────────────────┴────────────────────┴───┴───────────────────┴───────────────────┴──────────────────────┤
│ 3 rows                        

In [56]:
# テーブルをDescribe
duckdb.sql('DESCRIBE building_polygon')

┌──────────────┬─────────────┬─────────┬─────────┬─────────┬───────┐
│ column_name  │ column_type │  null   │   key   │ default │ extra │
│   varchar    │   varchar   │ varchar │ varchar │ varchar │ int32 │
├──────────────┼─────────────┼─────────┼─────────┼─────────┼───────┤
│ id           │ BIGINT      │ YES     │ NULL    │ NULL    │  NULL │
│ gml_id       │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ h3index10    │ UBIGINT     │ YES     │ NULL    │ NULL    │  NULL │
│ cal_zmin_m   │ DOUBLE      │ YES     │ NULL    │ NULL    │  NULL │
│ cal_area_m2  │ DOUBLE      │ YES     │ NULL    │ NULL    │  NULL │
│ cal_height_m │ DOUBLE      │ YES     │ NULL    │ NULL    │  NULL │
│ geom         │ GEOMETRY    │ YES     │ NULL    │ NULL    │  NULL │
└──────────────┴─────────────┴─────────┴─────────┴─────────┴───────┘

In [57]:
duckdb.sql('SELECT count(*) FROM building_polygon')

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│      1768280 │
└──────────────┘

In [58]:
duckdb.sql('select cal_zmin_m, cal_area_m2, cal_height_m from building_polygon').df()

,cal_zmin_m,cal_area_m2,cal_height_m
0,2.406,40.445644,6.735
1,2.792,53.061616,9.833
2,2.642,64.263192,6.605
3,2.483,39.170531,8.929
4,2.677,48.882327,6.083
...,...,...,...
1768275,2.666,61.282268,6.539
1768276,2.912,66.133218,7.058
1768277,2.780,50.205177,6.138
1768278,3.000,21.072562,2.236


import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_36, *['cal_zmin_m'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_37, *['cal_area_m2'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_38, *['cal_height_m'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(_df_39, *['cal_zmin_m'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(_df_40, *['cal_area_m2'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(_df_41, *['cal_height_m'], **{})
chart

import numpy as np
from google.colab import autoviz

def scatter_plots(df, colname_pairs, figscale=1, alpha=.8):
  from matplotlib import pyplot as plt
  plt.figure(figsize=(len(colname_pairs) * 6 * figscale, 6 * figscale))
  for plot_i, (x_colname, y_colname) in enumerate(colname_pairs, start=1):
    ax = plt.subplot(1, len(colname_pairs), plot_i)
    df.plot(kind='scatter', x=x_colname, y=y_colname, s=(32 * figscale), alpha=alpha, ax=ax)
    ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = scatter_plots(_df_42, *[[['cal_zmin_m', 'cal_area_m2'], ['cal_area_m2', 'cal_height_m']]], **{})
chart

In [28]:
# 標高データの読み込み（H3 level 10 のインデックスを含む）
elev_file = 'https://flateau.s3.ap-northeast-1.amazonaws.com/data/topography/tokyo23_elevation_h3lvl10.parquet'
duckdb.sql("CREATE OR REPLACE TABLE elevation AS SELECT h3index10, val_max, val_mean, val_median, val_mode, ST_GeomFromWKB(geometry) AS geom FROM '{}'".format(elev_file))


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [87]:
# 斜面傾斜データの読み込み（H3 level 10 のインデックスを含む）
slope_file = 'https://flateau.s3.ap-northeast-1.amazonaws.com/data/topography/tokyo23_slope_h3lvl10.parquet'
duckdb.sql("CREATE OR REPLACE TABLE slope AS SELECT h3index10, val_max, val_mean, val_median, val_mode, ST_GeomFromWKB(geometry) AS geom FROM '{}'".format(slope_file))


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [30]:
# 東京23区の標高分布
duckdb.sql('SELECT val_max, val_mean, val_median, val_mode FROM elevation').df()

,val_max,val_mean,val_median,val_mode
0,6.19,2.178936,3.040,0.60
1,6.20,2.296221,3.205,3.36
2,6.18,2.424900,3.260,3.41
3,3.55,2.315523,2.250,2.25
4,6.00,3.568281,3.460,3.38
...,...,...,...,...
46727,7.53,3.395628,2.970,2.30
46728,7.60,5.528303,6.885,2.33
46729,12.43,7.627472,7.260,7.29
46730,6.92,2.909865,2.400,2.32


In [52]:
duckdb.sql('SELECT val_max, val_mean, val_median, val_mode FROM slope').df()

,val_max,val_mean,val_median,val_mode
0,2.131054,0.610836,0.549421,0.298201
1,2.484312,0.704333,0.661217,0.654369
2,2.429445,0.844826,0.753571,0.469612
3,1.485541,0.495957,0.468135,0.075518
4,14.944903,1.953333,0.235748,0.111581
...,...,...,...,...
48651,1.653520,0.549164,0.524703,0.231044
48652,4.091684,1.030420,0.892122,0.572825
48653,NaN,NaN,NaN,NaN
48654,1.847704,0.770767,0.750116,0.405783


import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_27, *['val_max'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_28, *['val_mean'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_29, *['val_median'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_30, *['val_mode'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(_df_31, *['val_max'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(_df_32, *['val_mean'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(_df_33, *['val_median'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(_df_34, *['val_mode'], **{})
chart

import numpy as np
from google.colab import autoviz

def scatter_plots(df, colname_pairs, figscale=1, alpha=.8):
  from matplotlib import pyplot as plt
  plt.figure(figsize=(len(colname_pairs) * 6 * figscale, 6 * figscale))
  for plot_i, (x_colname, y_colname) in enumerate(colname_pairs, start=1):
    ax = plt.subplot(1, len(colname_pairs), plot_i)
    df.plot(kind='scatter', x=x_colname, y=y_colname, s=(32 * figscale), alpha=alpha, ax=ax)
    ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = scatter_plots(_df_35, *[[['val_max', 'val_mean'], ['val_mean', 'val_median'], ['val_median', 'val_mode']]], **{})
chart

## 空間クエリ101


### 緯度経度、ジオメトリの取り扱いについて基本的なことを確認
* geometry ST_Point(float x_lon, float y_lat);
* Tokyo Station (latitude:35.6812362, longitude:139.7645499)
* EPSG (European Petroleum Survey Group) コードについて、初めての人は以下を参照してください。
 * https://epsg.io/
 * https://epsg.org/home.html


In [24]:
#緯度、経度 (Google)
duckdb.sql("SELECT ST_POINT(35.6812362,139.7645499) geom")

┌────────────────────────────────┐
│              geom              │
│            geometry            │
├────────────────────────────────┤
│ POINT (35.6812362 139.7645499) │
└────────────────────────────────┘

In [25]:
#経度、緯度 (PostGIS: https://postgis.net/docs/ST_Point.html)
duckdb.sql("SELECT ST_POINT(139.7645499, 35.6812362) geom")

┌────────────────────────────────┐
│              geom              │
│            geometry            │
├────────────────────────────────┤
│ POINT (139.7645499 35.6812362) │
└────────────────────────────────┘

In [12]:
# ST_POINT() では、本来、経度,緯度 の順番のはずなのだが、、、そうするとうまく動かないので、、、、
duckdb.sql("SELECT ST_TRANSFORM(ST_POINT(35.6812362,139.7645499), 'EPSG:4326', 'EPSG:32654') geom")

┌─────────────────────────────────────────────┐
│                    geom                     │
│                  geometry                   │
├─────────────────────────────────────────────┤
│ POINT (388202.6519479657 3949296.928615559) │
└─────────────────────────────────────────────┘

In [17]:
# 経度・緯度のままだとうまくいかない
duckdb.sql("SELECT ST_TRANSFORM(ST_POINT(139.7645499, 35.6812362), 'EPSG:4326', 'EPSG:32654') geom")

┌───────────────────────────┐
│           geom            │
│         geometry          │
├───────────────────────────┤
│ POINT (Infinity Infinity) │
└───────────────────────────┘

In [16]:
# 経度・緯度をフリップして緯度・経度にしてから計算するとうまくいく
duckdb.sql("SELECT ST_TRANSFORM(ST_FlipCoordinates(ST_POINT(139.7645499,35.6812362)), 'EPSG:4326', 'EPSG:32654') geom")

┌─────────────────────────────────────────────┐
│                    geom                     │
│                  geometry                   │
├─────────────────────────────────────────────┤
│ POINT (388202.6519479657 3949296.928615559) │
└─────────────────────────────────────────────┘

In [15]:
# 東京と仙台の直線距離を計算。単位は、空間参照系の定義で決まります。この場合UTMなのでメートルが単位です。
duckdb.sql("""
SELECT ST_DISTANCE(
  ST_TRANSFORM(ST_POINT(35.6812362,139.7645499), 'EPSG:4326', 'EPSG:32654'),
  ST_TRANSFORM(ST_POINT(38.3140913,140.4391724), 'EPSG:4326', 'EPSG:32654')
) distance
""")

┌──────────────────┐
│     distance     │
│      double      │
├──────────────────┤
│ 298197.928068306 │
└──────────────────┘

In [27]:
# ちなみに東京と仙台の直線距離を計算。緯度経度を投影しないで計算させると、、、気をつけましょう。
duckdb.sql("""
SELECT ST_DISTANCE(
  ST_POINT(35.6812362,139.7645499),
  ST_POINT(38.3140913,140.4391724)
) distance
""")

┌───────────────────┐
│     distance      │
│      double       │
├───────────────────┤
│ 2.717911237531914 │
└───────────────────┘

In [21]:
# 東京駅からの直線距離を計算. XYを交換した後に計算。。。。
duckdb.sql("select geom, ST_TRANSFORM(ST_FlipCoordinates(geom), 'EPSG:4326', 'EPSG:6677') from building_polygon limit 3")


┌──────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────┐
│         geom         │               st_transform(st_flipcoordinates(geom), 'EPSG:4326', 'EPSG:6677')                │
│       geometry       │                                           geometry                                            │
├──────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────┤
│ POLYGON ((139.7123…  │ POLYGON ((-50851.20000016458 -10973.879999973837, -50854.140000166124 -10974.53999997561, -…  │
│ POLYGON ((139.7071…  │ POLYGON ((-50972.0100001638 -11441.21999997411, -50980.69500016411 -11445.509999973247, -50…  │
│ POLYGON ((139.7065…  │ POLYGON ((-50877.45000016399 -11500.25999997442, -50883.6750001647 -11504.729999973444, -50…  │
└──────────────────────┴───────────────────────────────────────────────────────────────────────────────────────────────┘

### H3インデックスを使って空間クエリ
DuckDB には、今の所ジオメトリに対して空間インデックスをPostGISのようには付与できないので、H3インデックスを明示的に使って空間結合する。

In [83]:
# 建物が立っている地形の最大傾斜の大きい順に上位10件の建物のリストを取得
query_01 = '''
  SELECT t1.gml_id, t1.geom, t2.val_median
  FROM building_polygon t1, slope t2
  WHERE t1.h3index10 = t2.h3index10
  ORDER BY val_max DESC
  LIMIT 20
'''
duckdb.sql(query_01)

┌──────────────────────┬──────────────────────────────────────────────────────────────────────────┬────────────────────┐
│        gml_id        │                                   geom                                   │     val_median     │
│       varchar        │                                 geometry                                 │       double       │
├──────────────────────┼──────────────────────────────────────────────────────────────────────────┼────────────────────┤
│ bldg_26811f7d-0a49…  │ POLYGON ((139.74449723500115 35.66740033892029, 139.74445959311024 35.…  │  8.298046588897705 │
│ bldg_3e9e4e15-3683…  │ POLYGON ((139.74477272466495 35.66700668755646, 139.7444687328894 35.6…  │  8.298046588897705 │
│ bldg_660a6fe6-dbd0…  │ POLYGON ((139.74373599923877 35.667795773549145, 139.74374646925907 35…  │  8.298046588897705 │
│ bldg_6d4b9ab2-db75…  │ POLYGON ((139.73544700956978 35.680677084621124, 139.73511680749166 35…  │  2.653118133544922 │
│ bldg_320b7641-aa4e…  │ POLYGON

### 距離を使った空間クエリ

In [81]:
# 東京駅から全ての23区の建物への直線距離を計算
duckdb.sql('''
  SELECT
    t1.gml_id,
    ST_DISTANCE(
      ST_TRANSFORM(ST_FlipCoordinates(geom), 'EPSG:4326', 'EPSG:6677'),
      ST_TRANSFORM(ST_POINT(35.6812362,139.7645499), 'EPSG:4326', 'EPSG:6677')
      ) distance
  FROM building_polygon t1
''')

┌───────────────────────────────────────────┬────────────────────┐
│                  gml_id                   │      distance      │
│                  varchar                  │       double       │
├───────────────────────────────────────────┼────────────────────┤
│ bldg_fc50c7d9-76ac-4576-bfbd-f37c74410928 │ 16199.534429526975 │
│ bldg_2e11a57e-c315-4351-8cca-22649d9763dc │ 16456.578249583657 │
│ bldg_72de059a-ddf0-4959-be00-db8108ca4a3a │ 16383.966347922305 │
│ bldg_2297d74f-b3d5-410a-b62c-a41f6913758c │   16387.7002811633 │
│ bldg_f6edafc1-b722-4b15-90e3-393cbb64d998 │ 16249.887186929942 │
│ bldg_c08f89da-8bf6-42f8-9d80-a45d7fea6a7f │  16430.63891037063 │
│ bldg_de0300aa-4958-41d2-adf7-f65282e67310 │ 16366.930487653508 │
│ bldg_cfeeaa46-c9c1-4e96-813d-8cacf91ad07c │ 16425.888308743528 │
│ bldg_a5f790cc-2b40-4717-a650-0b884dc74123 │ 16341.309520974342 │
│ bldg_1c6f6995-0609-4cd9-b989-7613f97e7829 │ 16468.704803247823 │
│                     ·                     │          ·      

In [79]:
# 東京駅から2km以内の建物の数、合計フットプリント面積、建物の高さの平均値を計算
# この場合、H3インデックスを使えないので、実行速度は遅い。H3ライブラリが使えれば最適化可能か。
%%time
duckdb.sql('''
  SELECT
    count(t1.gml_id) cnt,
    sum(t1.cal_area_m2) tot_area,
    avg(cal_height_m) avg_height
  FROM building_polygon t1
  WHERE
    ST_DISTANCE(
      ST_TRANSFORM(ST_FlipCoordinates(ST_CENTROID(geom)), 'EPSG:4326', 'EPSG:6677'),
      ST_TRANSFORM(ST_POINT(35.6812362,139.7645499), 'EPSG:4326', 'EPSG:6677')
    ) <= 2000
''')

CPU times: user 1.1 ms, sys: 0 ns, total: 1.1 ms
Wall time: 1.12 ms


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌───────┬───────────────────┬────────────────────┐
│  cnt  │     tot_area      │     avg_height     │
│ int64 │      double       │       double       │
├───────┼───────────────────┼────────────────────┤
│ 16811 │ 3808606.767950399 │ 19.898978525965276 │
└───────┴───────────────────┴────────────────────┘

In [80]:
# 同じ問題に対する別のアプローチ
duckdb.sql('''
  SELECT
    count(t1.gml_id) cnt,
    sum(t1.cal_area_m2) tot_area,
    avg(cal_height_m) avg_height
  FROM building_polygon t1
  WHERE
    ST_INTERSECTS(
      ST_TRANSFORM(ST_FlipCoordinates(ST_CENTROID(geom)), 'EPSG:4326', 'EPSG:6677'),
      ST_BUFFER(ST_TRANSFORM(ST_POINT(35.6812362,139.7645499), 'EPSG:4326', 'EPSG:6677'), 2000)
    )
''')


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌───────┬────────────────────┬────────────────────┐
│  cnt  │      tot_area      │     avg_height     │
│ int64 │       double       │       double       │
├───────┼────────────────────┼────────────────────┤
│ 16688 │ 3785685.1103719394 │ 19.920448525886957 │
└───────┴────────────────────┴────────────────────┘

### 最大傾斜が高く、標高の低いところにある建物のリスト

In [93]:
query_01 = '''
WITH tmp AS(
  SELECT t1.gml_id, t1.geom, t2.val_max, t2.h3index10
  FROM building_polygon t1, slope t2
  WHERE t1.h3index10 = t2.h3index10
  ORDER BY val_max DESC
  LIMIT 1000
)
SELECT tmp.gml_id, t1.val_median elev, tmp.val_max slope_max
FROM slope t1, tmp
WHERE t1.h3index10 = tmp.h3index10
'''
duckdb.sql(query_01)

InvalidInputException: ignored

### DuckDBのネイティブジオメトリタイプを試して見る

In [51]:
# DuckDBのネイティブ POINT_2D ジオメトリを追加してみる
duckdb.sql('ALTER TABLE building_polygon add geom_2dp POINT_2D')
duckdb.sql('update building_polygon set geom_2dp = ST_CENTROID(ST_FlipCoordinates(geom))::POINT_2D')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [75]:
# 東京駅から全ての23区の建物への直線距離を計算
%%time
duckdb.sql('''
  SELECT
    t1.gml_id,
    ST_DISTANCE(
      ST_TRANSFORM(geom_2dp, 'EPSG:4326', 'EPSG:6677'),
      ST_TRANSFORM(ST_POINT(35.6812362,139.7645499)::POINT_2D, 'EPSG:4326', 'EPSG:6677')
      ) distance
  FROM building_polygon t1
''')

CPU times: user 811 µs, sys: 0 ns, total: 811 µs
Wall time: 824 µs


┌───────────────────────────────────────────┬────────────────────┐
│                  gml_id                   │      distance      │
│                  varchar                  │       double       │
├───────────────────────────────────────────┼────────────────────┤
│ bldg_fc50c7d9-76ac-4576-bfbd-f37c74410928 │ 16202.130320538663 │
│ bldg_2e11a57e-c315-4351-8cca-22649d9763dc │  16461.84353808086 │
│ bldg_72de059a-ddf0-4959-be00-db8108ca4a3a │ 16388.850356952862 │
│ bldg_2297d74f-b3d5-410a-b62c-a41f6913758c │ 16391.534073012266 │
│ bldg_f6edafc1-b722-4b15-90e3-393cbb64d998 │ 16252.036103992812 │
│ bldg_c08f89da-8bf6-42f8-9d80-a45d7fea6a7f │ 16435.139746734392 │
│ bldg_de0300aa-4958-41d2-adf7-f65282e67310 │ 16370.600505466939 │
│ bldg_cfeeaa46-c9c1-4e96-813d-8cacf91ad07c │  16428.65350213206 │
│ bldg_a5f790cc-2b40-4717-a650-0b884dc74123 │ 16347.421510492963 │
│ bldg_1c6f6995-0609-4cd9-b989-7613f97e7829 │ 16478.707013098723 │
│                     ·                     │          ·      

In [77]:
# 東京駅から2km以内の建物の数、合計フットプリント面積、建物の高さの平均値を計算
%%time
duckdb.sql('''
  SELECT
    count(t1.gml_id) cnt,
    sum(t1.cal_area_m2) tot_area,
    avg(cal_height_m) avg_height
  FROM building_polygon t1
  WHERE
    ST_DISTANCE(
      ST_TRANSFORM(geom_2dp, 'EPSG:4326', 'EPSG:6677'),
      ST_TRANSFORM(ST_POINT(35.6812362,139.7645499)::POINT_2D, 'EPSG:4326', 'EPSG:6677')
    ) < 2000
''')

CPU times: user 918 µs, sys: 0 ns, total: 918 µs
Wall time: 933 µs


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌───────┬────────────────────┬───────────────────┐
│  cnt  │      tot_area      │    avg_height     │
│ int64 │       double       │      double       │
├───────┼────────────────────┼───────────────────┤
│ 16811 │ 3808606.7679503933 │ 19.89897852596522 │
└───────┴────────────────────┴───────────────────┘

## クエリの結果をGeoJSONで出力
[COPYコマンド](https://duckdb.org/docs/extensions/spatial#spatial-copy-functions)を使うと、クエリの結果をファイルとして出力できる。

In [19]:
# GeoJSONとしてクエリの結果を保存
duckdb.sql("copy ({}) to 'test.geojson' with (format gdal, driver 'GeoJSON', LAYER_CREATION_OPTIONS 'WRITE_BBOX=YES')".format(query_01))


## クエリの結果をDataFrameに格納

In [35]:
# change to DF
df = duckdb.sql('select gml_id, cal_height_m, ST_AsText(geom) geom_txt from building_polygon limit 500').df()
df

,gml_id,cal_height_m,geom_txt
0,bldg_fc50c7d9-76ac-4576-bfbd-f37c74410928,6.735,POLYGON ((139.7123070884317 35.541587725969336...
1,bldg_2e11a57e-c315-4351-8cca-22649d9763dc,9.833,POLYGON ((139.70715471131086 35.54049347329776...
2,bldg_72de059a-ddf0-4959-be00-db8108ca4a3a,6.605,POLYGON ((139.70650225272118 35.54134514528564...
3,bldg_2297d74f-b3d5-410a-b62c-a41f6913758c,8.929,"POLYGON ((139.706557904104 35.541302207399234,..."
4,bldg_f6edafc1-b722-4b15-90e3-393cbb64d998,6.083,POLYGON ((139.71058239366005 35.54154636660162...
...,...,...,...
495,bldg_f9dd69da-d3b8-43a9-8b6e-9198c70d6fa4,6.637,POLYGON ((139.70707248053844 35.54271892419017...
496,bldg_1d96d223-40c5-4804-bde5-78e9e5cfe35b,6.766,POLYGON ((139.70470434678694 35.54861855219195...
497,bldg_38fba4dd-4ab1-478d-a930-05a6c1506107,3.343,POLYGON ((139.7063953013518 35.548537596388506...
498,bldg_23e83c98-cb5c-4ebc-b0e8-bc577894ad12,5.464,POLYGON ((139.71235882592032 35.54811457430077...


## クエリの結果をGeoDataFrameに格納

In [36]:
# change to GDF
df["geom"] = gpd.GeoSeries.from_wkt(df["geom_txt"])
gdf = gpd.GeoDataFrame(df, geometry="geom")
gdf

,gml_id,cal_height_m,geom_txt,geom
0,bldg_fc50c7d9-76ac-4576-bfbd-f37c74410928,6.735,POLYGON ((139.7123070884317 35.541587725969336...,"POLYGON ((139.71231 35.54159, 139.71230 35.541..."
1,bldg_2e11a57e-c315-4351-8cca-22649d9763dc,9.833,POLYGON ((139.70715471131086 35.54049347329776...,"POLYGON ((139.70715 35.54049, 139.70711 35.540..."
2,bldg_72de059a-ddf0-4959-be00-db8108ca4a3a,6.605,POLYGON ((139.70650225272118 35.54134514528564...,"POLYGON ((139.70650 35.54135, 139.70645 35.541..."
3,bldg_2297d74f-b3d5-410a-b62c-a41f6913758c,8.929,"POLYGON ((139.706557904104 35.541302207399234,...","POLYGON ((139.70656 35.54130, 139.70652 35.541..."
4,bldg_f6edafc1-b722-4b15-90e3-393cbb64d998,6.083,POLYGON ((139.71058239366005 35.54154636660162...,"POLYGON ((139.71058 35.54155, 139.71059 35.541..."
...,...,...,...,...
495,bldg_f9dd69da-d3b8-43a9-8b6e-9198c70d6fa4,6.637,POLYGON ((139.70707248053844 35.54271892419017...,"POLYGON ((139.70707 35.54272, 139.70707 35.542..."
496,bldg_1d96d223-40c5-4804-bde5-78e9e5cfe35b,6.766,POLYGON ((139.70470434678694 35.54861855219195...,"POLYGON ((139.70470 35.54862, 139.70468 35.548..."
497,bldg_38fba4dd-4ab1-478d-a930-05a6c1506107,3.343,POLYGON ((139.7063953013518 35.548537596388506...,"POLYGON ((139.70640 35.54854, 139.70639 35.548..."
498,bldg_23e83c98-cb5c-4ebc-b0e8-bc577894ad12,5.464,POLYGON ((139.71235882592032 35.54811457430077...,"POLYGON ((139.71236 35.54811, 139.71235 35.548..."


## 地図表示

### leafmapを利用して、データを2D地図表示

In [23]:
# visualize data with 2D leafmap
m = leafmap.Map(center=(35.5744826,139.7078659), zoom=13)
m.add_gdf(gdf, layer_name="Plateau Buildings")
m

Map(center=[35.5744826, 139.7078659], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_titl…

### leafmap/deck.gl を利用して、データを3D地図表示

In [42]:
# visualize data in 3D with deck.gl
import leafmap.deck as leafmap

initial_view_state = {
    "latitude": 35.5744826,
    "longitude": 139.7078659,
    "zoom": 13,
    "pitch": 45,
    "bearing": 10,
}

m = leafmap.Map(initial_view_state=initial_view_state)

m.add_gdf(
    gdf,
    extruded=True,
    get_elevation="cal_height_m",
    elevation_range=[0, 3000],
    elevation_scale=1,
)
m

<IPython.core.display.Javascript object>

{
  "initialViewState": {
    "bearing": 10,
    "latitude": 35.5744826,
    "longitude": 139.7078659,
    "pitch": 45,
    "zoom": 13
  },
  "layers": [
    {
      "@@type": "GeoJsonLayer",
      "data": [
        {
          "cal_height_m": 6.734999999999999,
          "geom_txt": "POLYGON ((139.7123070884317 35.541587725969336, 139.71229984940234 35.54156121777629, 139.7122439192179 35.541571437500146, 139.7122379969294 35.54154939251641, 139.7121992761594 35.54155638448747, 139.7122032244802 35.54157099100549, 139.71218932484814 35.541573410802144, 139.71219886845319 35.54160749301028, 139.7123070884317 35.541587725969336))",
          "geometry": {
            "coordinates": [
              [
                [
                  139.7123070884317,
                  35.541587725969336
                ],
                [
                  139.71229984940234,
                  35.54156121777629
                ],
                [
                  139.7122439192179,
                  35.541571437500146
                ],
                [
                  139.7122379969294,
                  35.54154939251641
                ],
                [
                  139.7121992761594,
                  35.54155638448747
                ],
                [
                  139.7122032244802,
                  35.54157099100549
                ],
                [
                  139.71218932484814,
                  35.541573410802144
                ],
                [
                  139.71219886845319,
                  35.54160749301028
                ],
                [
                  139.7123070884317,
                  35.541587725969336
                ]
              ]
            ],
            "type": "Polygon"
          },
          "gml_id": "bldg_fc50c7d9-76ac-4576-bfbd-f37c74410928"
        },
        {
          "cal_height_m": 9.833,
          "geom_txt": "POLYGON ((139.70715471131086 35.540493473297765, 139.70710752213267 35.54041513800132, 139.70704958663651 35.540438333267254, 139.70708555434518 35.5404990796912, 139.70710343158214 35.54049193233668, 139.70711432152413 35.540509926509365, 139.70715471131086 35.540493473297765))",
          "geometry": {
            "coordinates": [
              [
                [
                  139.70715471131086,
                  35.540493473297765
                ],
                [
                  139.70710752213267,
                  35.54041513800132
                ],
                [
                  139.70704958663651,
                  35.540438333267254
                ],
                [
                  139.70708555434518,
                  35.5404990796912
                ],
                [
                  139.70710343158214,
                  35.54049193233668
                ],
                [
                  139.70711432152413,
                  35.540509926509365
                ],
                [
                  139.70715471131086,
                  35.540493473297765
                ]
              ]
            ],
            "type": "Polygon"
          },
          "gml_id": "bldg_2e11a57e-c315-4351-8cca-22649d9763dc"
        },
        {
          "cal_height_m": 6.605,
          "geom_txt": "POLYGON ((139.70650225272118 35.541345145285646, 139.70645304362753 35.541288981865286, 139.7063778700233 35.541332980823725, 139.7064270790912 35.54138914427429, 139.70650225272118 35.541345145285646))",
          "geometry": {
            "coordinates": [
              [
                [
                  139.70650225272118,
                  35.541345145285646
                ],
                [
                  139.70645304362753,
                  35.541288981865286
                ],
                [
                  139.7063778700233,
                  35.541332980823725
                ],
                [
                  139.7064270790912,
                  35.54138914427429
                ],
                [